In [29]:


# Load the dataset files

import numpy as np

# Load dataset files
synthetic_eeg_eog_path = "/home/tulgaa/Desktop/denoisenet/Linear_Mixing/EEG+EOG/Linear_synthetic_eeg_eog.npy"
clean_eeg_path = "/home/tulgaa/Desktop/denoisenet/Linear_Mixing/EEG+EOG/EEG_all_epochs.npy"

# Load the numpy arrays
synthetic_eeg_eog = np.load(synthetic_eeg_eog_path, allow_pickle=True).item()  # Dictionary of SNR levels
clean_eeg = np.load(clean_eeg_path)  # Ground truth EEG

# Inspect dataset keys and shapes
print("🔹 Synthetic EEG+EOG Dictionary Keys (SNR Levels):", synthetic_eeg_eog.keys())
print("🔹 Clean EEG Shape:", clean_eeg.shape)

# Print shape of each SNR level data
for snr in synthetic_eeg_eog.keys():
    print(f"🔹 SNR Level {snr}: Shape = {synthetic_eeg_eog[snr].shape}")

# Check statistics of clean EEG data
print("🔹 Clean EEG Mean:", np.mean(clean_eeg))
print("🔹 Clean EEG Std Dev:", np.std(clean_eeg))


🔹 Synthetic EEG+EOG Dictionary Keys (SNR Levels): dict_keys([-7, -6, -5, -4, -3, -2, -1, 0, 1, 2])
🔹 Clean EEG Shape: (3400, 512)
🔹 SNR Level -7: Shape = (3400, 512)
🔹 SNR Level -6: Shape = (3400, 512)
🔹 SNR Level -5: Shape = (3400, 512)
🔹 SNR Level -4: Shape = (3400, 512)
🔹 SNR Level -3: Shape = (3400, 512)
🔹 SNR Level -2: Shape = (3400, 512)
🔹 SNR Level -1: Shape = (3400, 512)
🔹 SNR Level 0: Shape = (3400, 512)
🔹 SNR Level 1: Shape = (3400, 512)
🔹 SNR Level 2: Shape = (3400, 512)
🔹 Clean EEG Mean: -0.16774763156951383
🔹 Clean EEG Std Dev: 231.82429071340553


In [30]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Step 1: Expand clean EEG dataset (repeat 10 times to match noisy EEG dataset)
X_clean = np.repeat(clean_eeg, 10, axis=0)  # Shape becomes (34,000, 512)

# Step 2: Stack all noisy EEG signals across different SNR levels
X_noisy = np.concatenate([synthetic_eeg_eog[snr] for snr in synthetic_eeg_eog.keys()], axis=0)  # Shape (34,000, 512)

# Step 3: Normalize both clean and noisy EEG signals to the range [-1, 1]
scaler = MinMaxScaler(feature_range=(-1, 1))
X_clean_scaled = scaler.fit_transform(X_clean)  # Normalize clean EEG
X_noisy_scaled = scaler.transform(X_noisy)  # Normalize noisy EEG

# Print dataset details
print(f"✅ Final dataset shapes: Noisy EEG {X_noisy_scaled.shape}, Clean EEG {X_clean_scaled.shape}")
print(f"✅ Clean EEG Mean after Scaling: {np.mean(X_clean_scaled):.4f}, Std Dev: {np.std(X_clean_scaled):.4f}")
print(f"✅ Noisy EEG Mean after Scaling: {np.mean(X_noisy_scaled):.4f}, Std Dev: {np.std(X_noisy_scaled):.4f}")


✅ Final dataset shapes: Noisy EEG (34000, 512), Clean EEG (34000, 512)
✅ Clean EEG Mean after Scaling: -0.0066, Std Dev: 0.2461
✅ Noisy EEG Mean after Scaling: -0.0070, Std Dev: 0.6368


In [31]:
from sklearn.model_selection import train_test_split

# Initialize lists to store train/test sets
X_train_list, X_test_list, y_train_list, y_test_list = [], [], [], []

# Split each SNR level separately to preserve SNR distribution
for snr in synthetic_eeg_eog.keys():
    noisy_snr = synthetic_eeg_eog[snr]  # Get noisy EEG data for this SNR level
    clean_snr = clean_eeg  # Corresponding clean EEG data

    # Split into 80% Train, 20% Test
    X_train_snr, X_test_snr, y_train_snr, y_test_snr = train_test_split(
        noisy_snr, clean_snr, test_size=0.2, random_state=42
    )

    # Store the results
    X_train_list.append(X_train_snr)
    X_test_list.append(X_test_snr)
    y_train_list.append(y_train_snr)
    y_test_list.append(y_test_snr)

# Stack all train and test data together
X_train = np.vstack(X_train_list)
X_test = np.vstack(X_test_list)
y_train = np.vstack(y_train_list)
y_test = np.vstack(y_test_list)

# Print dataset sizes
print(f"✅ Final Train Set: Noisy EEG {X_train.shape}, Clean EEG {y_train.shape}")
print(f"✅ Final Test Set: Noisy EEG {X_test.shape}, Clean EEG {y_test.shape}")


✅ Final Train Set: Noisy EEG (27200, 512), Clean EEG (27200, 512)
✅ Final Test Set: Noisy EEG (6800, 512), Clean EEG (6800, 512)


In [33]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define the EEGDiR LSTM Model
class EEGDiR_LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_size=128, num_layers=2, dropout=0.3):
        super(EEGDiR_LSTM, self).__init__()

        # LSTM Layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)

        # Fully Connected Layers
        self.fc1 = nn.Linear(hidden_size, 64)  
        self.fc2 = nn.Linear(64, 32)  
        self.fc3 = nn.Linear(32, input_size)  

        # Activation function
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)  
        x = self.relu(self.fc1(lstm_out))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.fc3(x)  # Output the denoised EEG
        return x

# Initialize the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EEGDiR_LSTM().to(device)

# Define Loss and Optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print(f"✅ Model is initialized on {device}")


✅ Model is initialized on cuda


In [15]:
import torch
from torch.utils.data import Dataset, DataLoader

# Custom PyTorch Dataset
class EEGDataset(Dataset):
    def __init__(self, noisy_data, clean_data):
        self.noisy_data = torch.tensor(noisy_data, dtype=torch.float32).unsqueeze(-1)  # (N, 512, 1)
        self.clean_data = torch.tensor(clean_data, dtype=torch.float32).unsqueeze(-1)  # (N, 512, 1)

    def __len__(self):
        return len(self.noisy_data)

    def __getitem__(self, idx):
        return self.noisy_data[idx], self.clean_data[idx]

# Define batch size
batch_size = 1024

# Create DataLoader for training and testing
train_dataset = EEGDataset(X_train, y_train)
test_dataset = EEGDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"✅ DataLoaders are ready! Training Batch Size: {batch_size}")


✅ DataLoaders are ready! Training Batch Size: 1024


In [34]:
import torch.optim.lr_scheduler as lr_scheduler

# Function to train the model with Early Stopping
def train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=50, patience=3):
    model.train()  # Set model to training mode

    best_loss = float("inf")
    early_stop_counter = 0

    # Learning Rate Scheduler (Reduce LR if loss plateaus)
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=2, factor=0.5, verbose=True)

    for epoch in range(num_epochs):
        total_train_loss = 0.0
        total_test_loss = 0.0

        # Training loop
        for noisy_batch, clean_batch in train_loader:
            noisy_batch, clean_batch = noisy_batch.to(device), clean_batch.to(device)

            optimizer.zero_grad()
            outputs = model(noisy_batch)
            loss = criterion(outputs, clean_batch)
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()

        # Validation loop
        model.eval()
        with torch.no_grad():
            for noisy_batch, clean_batch in test_loader:
                noisy_batch, clean_batch = noisy_batch.to(device), clean_batch.to(device)
                outputs = model(noisy_batch)
                loss = criterion(outputs, clean_batch)
                total_test_loss += loss.item()

        model.train()

        # Calculate average losses
        avg_train_loss = total_train_loss / len(train_loader)
        avg_test_loss = total_test_loss / len(test_loader)

        # Learning rate scheduling
        scheduler.step(avg_test_loss)

        # Print loss for monitoring
        print(f"🔹 Epoch [{epoch+1}/{num_epochs}] | Train Loss: {avg_train_loss:.6f} | Test Loss: {avg_test_loss:.6f}")

        # Check for early stopping
        if avg_test_loss < best_loss:
            best_loss = avg_test_loss
            early_stop_counter = 0  # Reset counter
            torch.save(model.state_dict(), "best_lstm_model.pth")  # Save best model
            print("✅ Model Improved! Saving...")
        else:
            early_stop_counter += 1

        if early_stop_counter >= patience:
            print("⏹️ Early Stopping Triggered. Training Stopped!")
            break

    print("✅ Training Completed!")

# Start Training with Early Stopping
train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=50, patience=3)


/home/tulgaa/anaconda3/envs/medsam/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


🔹 Epoch [1/50] | Train Loss: 51942.156105 | Test Loss: 47763.353237
✅ Model Improved! Saving...
🔹 Epoch [2/50] | Train Loss: 39783.776910 | Test Loss: 30532.616908
✅ Model Improved! Saving...
🔹 Epoch [3/50] | Train Loss: 25306.262297 | Test Loss: 20927.696708
✅ Model Improved! Saving...
🔹 Epoch [4/50] | Train Loss: 20844.667535 | Test Loss: 18609.704241
✅ Model Improved! Saving...
🔹 Epoch [5/50] | Train Loss: 19159.754630 | Test Loss: 17776.962472
✅ Model Improved! Saving...
🔹 Epoch [6/50] | Train Loss: 18051.425130 | Test Loss: 17040.471959
✅ Model Improved! Saving...
🔹 Epoch [7/50] | Train Loss: 17171.120515 | Test Loss: 16524.702846
✅ Model Improved! Saving...
🔹 Epoch [8/50] | Train Loss: 16748.587059 | Test Loss: 16428.017160
✅ Model Improved! Saving...
🔹 Epoch [9/50] | Train Loss: 16245.704246 | Test Loss: 16235.419224
✅ Model Improved! Saving...
🔹 Epoch [10/50] | Train Loss: 15924.886936 | Test Loss: 15902.839704
✅ Model Improved! Saving...
🔹 Epoch [11/50] | Train Loss: 15439.057

In [35]:
import numpy as np
from scipy.stats import pearsonr
from scipy.fftpack import fft

# Load the best trained model
model.load_state_dict(torch.load("best_lstm_model.pth"))
model.eval()

# Function to compute RRMSE (Relative Root Mean Square Error)
def compute_rrmse(denoised, clean):
    return np.sqrt(np.mean((denoised - clean) ** 2)) / np.sqrt(np.mean(clean ** 2))

# Function to compute RRMSE in the Spectrum Domain (RRMSE-S)
def compute_rrmse_spectrum(denoised, clean):
    fft_clean = np.abs(fft(clean))  # Compute FFT
    fft_denoised = np.abs(fft(denoised))
    return compute_rrmse(fft_denoised, fft_clean)

# Function to compute Correlation Coefficient (CC)
def compute_cc(denoised, clean):
    return pearsonr(denoised.flatten(), clean.flatten())[0]

# Store results per SNR level
snr_metrics = {}

# Loop through each SNR level
with torch.no_grad():
    for snr in synthetic_eeg_eog.keys():
        print(f"🔹 Evaluating for SNR Level: {snr}")

        # Get test data for this SNR level
        noisy_snr = synthetic_eeg_eog[snr][-int(0.2 * len(synthetic_eeg_eog[snr])):]  # Take 20% as test set
        clean_snr = clean_eeg[-int(0.2 * len(clean_eeg)):]  # Corresponding clean EEG

        # Convert to PyTorch tensors
        noisy_tensor = torch.tensor(noisy_snr, dtype=torch.float32).unsqueeze(-1).to(device)
        clean_tensor = torch.tensor(clean_snr, dtype=torch.float32).unsqueeze(-1).to(device)

        # Get model predictions
        denoised_tensor = model(noisy_tensor).cpu().numpy().squeeze()
        clean_snr_np = clean_tensor.cpu().numpy().squeeze()

        # Compute metrics
        rrmse_t_list = [compute_rrmse(denoised_tensor[i], clean_snr_np[i]) for i in range(len(clean_snr_np))]
        rrmse_s_list = [compute_rrmse_spectrum(denoised_tensor[i], clean_snr_np[i]) for i in range(len(clean_snr_np))]
        cc_list = [compute_cc(denoised_tensor[i], clean_snr_np[i]) for i in range(len(clean_snr_np))]

        # Store results for this SNR level
        snr_metrics[snr] = {
            "RRMSE-T": np.mean(rrmse_t_list),
            "RRMSE-S": np.mean(rrmse_s_list),
            "CC": np.mean(cc_list)
        }

# Print Results in Table Format
print("\n🔹 **Final Evaluation Per SNR Level:**")
print("-----------------------------------------------------")
print("| SNR  |  RRMSE-T  |  RRMSE-S  |   CC   |")
print("-----------------------------------------------------")
for snr in sorted(snr_metrics.keys()):
    print(f"| {snr:3d}  |  {snr_metrics[snr]['RRMSE-T']:.6f}  |  {snr_metrics[snr]['RRMSE-S']:.6f}  |  {snr_metrics[snr]['CC']:.6f}  |")
print("-----------------------------------------------------")


🔹 Evaluating for SNR Level: -7
🔹 Evaluating for SNR Level: -6
🔹 Evaluating for SNR Level: -5
🔹 Evaluating for SNR Level: -4
🔹 Evaluating for SNR Level: -3
🔹 Evaluating for SNR Level: -2
🔹 Evaluating for SNR Level: -1
🔹 Evaluating for SNR Level: 0
🔹 Evaluating for SNR Level: 1
🔹 Evaluating for SNR Level: 2

🔹 **Final Evaluation Per SNR Level:**
-----------------------------------------------------
| SNR  |  RRMSE-T  |  RRMSE-S  |   CC   |
-----------------------------------------------------
|  -7  |  0.925227  |  0.633353  |  0.612503  |
|  -6  |  0.797495  |  0.533744  |  0.685796  |
|  -5  |  0.689837  |  0.457695  |  0.750957  |
|  -4  |  0.604933  |  0.403349  |  0.803887  |
|  -3  |  0.539153  |  0.363159  |  0.843665  |
|  -2  |  0.493292  |  0.336257  |  0.870625  |
|  -1  |  0.459594  |  0.316222  |  0.888859  |
|   0  |  0.435934  |  0.302739  |  0.901197  |
|   1  |  0.422188  |  0.293768  |  0.908183  |
|   2  |  0.415177  |  0.289460  |  0.912095  |
------------------------

In [36]:
import numpy as np
from scipy.stats import pearsonr
from scipy.fftpack import fft

# Load the best trained model
model.load_state_dict(torch.load("best_lstm_model.pth"))
model.eval()

# Function to compute RRMSE (Relative Root Mean Square Error)
def compute_rrmse(denoised, clean):
    return np.sqrt(np.mean((denoised - clean) ** 2)) / np.sqrt(np.mean(clean ** 2))

# Function to compute RRMSE in the Spectrum Domain (RRMSE-S)
def compute_rrmse_spectrum(denoised, clean):
    fft_clean = np.abs(fft(clean))  # Compute FFT
    fft_denoised = np.abs(fft(denoised))
    return compute_rrmse(fft_denoised, fft_clean)

# Function to compute Correlation Coefficient (CC)
def compute_cc(denoised, clean):
    return pearsonr(denoised.flatten(), clean.flatten())[0]

# Store results per SNR level
snr_metrics = {}
all_rrmse_t = []
all_rrmse_s = []
all_cc = []

# Loop through each SNR level
with torch.no_grad():
    for snr in synthetic_eeg_eog.keys():
        print(f"🔹 Evaluating for SNR Level: {snr}")

        # Get test data for this SNR level
        noisy_snr = synthetic_eeg_eog[snr][-int(0.2 * len(synthetic_eeg_eog[snr])):]  # Take 20% as test set
        clean_snr = clean_eeg[-int(0.2 * len(clean_eeg)):]  # Corresponding clean EEG

        # Convert to PyTorch tensors
        noisy_tensor = torch.tensor(noisy_snr, dtype=torch.float32).unsqueeze(-1).to(device)
        clean_tensor = torch.tensor(clean_snr, dtype=torch.float32).unsqueeze(-1).to(device)

        # Get model predictions
        denoised_tensor = model(noisy_tensor).cpu().numpy().squeeze()
        clean_snr_np = clean_tensor.cpu().numpy().squeeze()

        # Compute metrics
        rrmse_t_list = [compute_rrmse(denoised_tensor[i], clean_snr_np[i]) for i in range(len(clean_snr_np))]
        rrmse_s_list = [compute_rrmse_spectrum(denoised_tensor[i], clean_snr_np[i]) for i in range(len(clean_snr_np))]
        cc_list = [compute_cc(denoised_tensor[i], clean_snr_np[i]) for i in range(len(clean_snr_np))]

        # Store results for this SNR level
        snr_metrics[snr] = {
            "RRMSE-T": np.mean(rrmse_t_list),
            "RRMSE-S": np.mean(rrmse_s_list),
            "CC": np.mean(cc_list)
        }

        # Collect all values for averaging
        all_rrmse_t.extend(rrmse_t_list)
        all_rrmse_s.extend(rrmse_s_list)
        all_cc.extend(cc_list)

# Compute overall averages
avg_rrmse_t = np.mean(all_rrmse_t)
avg_rrmse_s = np.mean(all_rrmse_s)
avg_cc = np.mean(all_cc)

# Print Results in Table Format
print("\n🔹 **Final Evaluation Per SNR Level:**")
print("-----------------------------------------------------")
print("| SNR  |  RRMSE-T  |  RRMSE-S  |   CC   |")
print("-----------------------------------------------------")
for snr in sorted(snr_metrics.keys()):
    print(f"| {snr:3d}  |  {snr_metrics[snr]['RRMSE-T']:.6f}  |  {snr_metrics[snr]['RRMSE-S']:.6f}  |  {snr_metrics[snr]['CC']:.6f}  |")
print("-----------------------------------------------------")
print(f"🔹 **Overall Averages**: RRMSE-T: {avg_rrmse_t:.6f}, RRMSE-S: {avg_rrmse_s:.6f}, CC: {avg_cc:.6f}")


🔹 Evaluating for SNR Level: -7
🔹 Evaluating for SNR Level: -6
🔹 Evaluating for SNR Level: -5
🔹 Evaluating for SNR Level: -4
🔹 Evaluating for SNR Level: -3
🔹 Evaluating for SNR Level: -2
🔹 Evaluating for SNR Level: -1
🔹 Evaluating for SNR Level: 0
🔹 Evaluating for SNR Level: 1
🔹 Evaluating for SNR Level: 2

🔹 **Final Evaluation Per SNR Level:**
-----------------------------------------------------
| SNR  |  RRMSE-T  |  RRMSE-S  |   CC   |
-----------------------------------------------------
|  -7  |  0.925227  |  0.633353  |  0.612503  |
|  -6  |  0.797495  |  0.533744  |  0.685796  |
|  -5  |  0.689837  |  0.457695  |  0.750957  |
|  -4  |  0.604933  |  0.403349  |  0.803887  |
|  -3  |  0.539153  |  0.363159  |  0.843665  |
|  -2  |  0.493292  |  0.336257  |  0.870625  |
|  -1  |  0.459594  |  0.316222  |  0.888859  |
|   0  |  0.435934  |  0.302739  |  0.901197  |
|   1  |  0.422188  |  0.293768  |  0.908183  |
|   2  |  0.415177  |  0.289460  |  0.912095  |
------------------------